In [ ]:
import h5py
from sklearn.model_selection import train_test_split
import numpy as np
#from getdata import load
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import hamming_loss
from keras import backend as K
import cv2
import scipy.io as sio
import os
from skimage import io
from skimage.transform import resize
import pandas as pd
K.set_image_dim_ordering('th')


In [ ]:
train_photos = pd.read_csv('D:/Share_Folder/Pace_3rd_Sem/Big_Data_Analytics/Project Files/train_photo_to_biz_ids.csv',
                           names=["photo_id","business_id"],header=None)
train_attr = pd.read_csv('D:/Share_Folder/Pace_3rd_Sem/Big_Data_Analytics/Project Files/train.csv',
                         header=None,names=["business_id","labels"])
train_id = pd.read_csv('D:/Share_Folder/Pace_3rd_Sem/Big_Data_Analytics/Project Files/train_photo_to_biz_ids.csv',
                       header=None,names=["photo_id","business_id"]) 
test_photos = pd.read_csv('D:/Share_Folder/Pace_3rd_Sem/Big_Data_Analytics/Project Files/test_photo_to_biz.csv',
                          header=None,names=["photo_id","business_id"])

append = pd.read_csv('D:/Share_Folder/Pace_3rd_Sem/Big_Data_Analytics/Project Files/append.csv',
                     header=None, names=["photo_id","business_id","labels"])


In [ ]:
append_X = append.photo_id
append_y = append['labels']


In [ ]:
np.random.seed(456)
indices=np.random.permutation(len(append))
x_train=append_X[indices[:187873]]
y_train=append_y[indices[:187873]]
x_val=append_X[indices[187873:211357]]
y_val=append_y[indices[187873:211357]]
x_test=append_X[indices[211357:]]
y_test=append_y[indices[211357:]]

In [ ]:
train_yelp = pd.DataFrame({'photo_id':x_train,'labels':y_train})
val_yelp = pd.DataFrame({'photo_id':x_val,'labels':y_val})
test_yelp = pd.DataFrame({'photo_id':x_test,'labels':y_test})

In [ ]:
#train_yelp.head()
train_yelp.to_csv('train_yelp.csv',sep='\t',encoding='utf-8')
val_yelp.to_csv('val_yelp.csv',sep='\t',encoding='utf-8')
test_yelp.to_csv('test_yelp.csv',sep='\t',encoding='utf-8')

In [ ]:
#y=append['labels'].str.get_dummies(sep=' ')
y_train_vect = y_train.str.get_dummies(sep=' ')
y_val_vect = y_val.str.get_dummies(sep=' ')
y_test_vect = y_test.str.get_dummies(sep=' ')

In [ ]:
x_train_vect = []

for i in x_train:
    a= int(i)
    #print ("reading image:"+str(a) + ".jpg")
    img = ('D:/Share_Folder/Pace_3rd_Sem/Big_Data_Analytics/Project Files/train_photos/'  + str(a) + ".jpg")
    img = cv2.imread(img)
    img = cv2.resize(img,(100,100))
    img = img.transpose((2,0,1))
    # img = io.imread(img)
    # img = resize(img,(100,100))
    # img = img.transpose()
    x_train_vect.append(img)

x_train_vect = np.array(x_train_vect)

f = h5py.File("trainyelp.h5")
f['x'] = x_train_vect
f['y'] = y_train_vect
f.close()

In [ ]:
train = h5py.File("trainyelp.h5","r")
x_train = train['x'].value
y_train = train['y'].value
train.close()


In [ ]:
val = h5py.File("valyelp.h5","r")
x_val = val['x'].value
y_val = val['y'].value
val.close()

In [ ]:
x_train_float = x_train.astype('float32')
x_val_float  = x_val.astype('float32')
#x_test = x_test.astype('float32')

x_train_float /= 255
x_val_float /= 255
#x_test /= 255


In [ ]:
len(x_train_float),len(y_train)


In [ ]:
len(x_val_float),len(y_val)


In [ ]:
from keras.optimizers import Adam

In [ ]:
model = Sequential()
model.add(Convolution2D(32, kernel_size=(5, 5),padding='same',input_shape=(3 , 100, 100)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

'''model.add(Convolution2D(64,(3, 3), padding='same'))
model.add(Activation('relu'))'''
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(9))
model.add(Activation('sigmoid'))




In [ ]:
# let's train the model using SGD + momentum (how original).
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
ada = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=ada, metrics=['accuracy'])

# model.load_weights("weights.16-0.86800.hdf5")

In [ ]:
check = ModelCheckpoint("weights.{epoch:02d}-{val_acc:.5f}.hdf5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='auto')